In [ ]:
#default_exp dataloader

# Dataloader

> implement a NumpyDataLoader for speedup

reimplement tsai functionality for numpy dataloader

In [ ]:
#using the NumpyDataLoader in tsai is significantly faster than using the fastaiv2 DataLoader, 
#maybe build a custom dataloader to optimize for odds data



# import tsai
# from tsai.all import *

In [ ]:
#export
import numpy as np
import scipy as sp
#import torch
from fastai2.torch_basics import *
from fastai2.data.all import *
from fastai2.callback.all import *

from fastai2.data.all import *
from fastai2.basics import *

In [ ]:
#export
import psutil
import fastai2
import fastcore
import torch

In [ ]:
#export
from torchtools.data import *
from torchtools.datasets import *
from torchtools.augmentations import *
from torchtools.datablock import *

In [ ]:
#export
from torchtools.models import *
from torchtools.core import *

import pandas as pd
import numpy as np
from functools import partial

_verbose=True

#### Utils

In [ ]:
#export
#tsai.imports
cpus = defaults.cpus
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
#export
#tsai.utils
def bytes2GB(byts):
    return round(byts / math.pow(1024, 3), 2)

In [ ]:
# print('tsai       :', tsai.__version__)
print('fastai2    :', fastai2.__version__)
print('fastcore   :', fastcore.__version__)
print('torch      :', torch.__version__)
print('scipy      :', sp.__version__)
print('numpy      :', np.__version__)
print('pandas     :', pd.__version__)
print(f'Total RAM  : {bytes2GB(psutil.virtual_memory().total):5.2f} GB')
print(f'Used RAM   : {bytes2GB(psutil.virtual_memory().used):5.2f} GB')
print('n_cpus     :', cpus)
iscuda = torch.cuda.is_available()
if iscuda: print('device     : {} ({})'.format(device, torch.cuda.get_device_name(0)))
else: print('device     :', device)

fastai2    : 0.0.18
fastcore   : 0.1.18
torch      : 1.3.1
scipy      : 1.4.1
numpy      : 1.18.1
pandas     : 0.25.3
Total RAM  : 15.56 GB
Used RAM   :  9.04 GB
n_cpus     : 4
device     : cuda (GeForce GTX 950)


In [ ]:
#export
#tsai.utils
#export
def totensor(o):
    if isinstance(o, torch.Tensor): return o
    elif isinstance(o, np.ndarray):  return torch.from_numpy(o)
    assert False, f"Can't convert {type(o)} to torch.Tensor"


def toarray(o):
    if isinstance(o, np.ndarray): return o
    elif isinstance(o, torch.Tensor): return o.cpu().numpy()
    assert False, f"Can't convert {type(o)} to np.array"


def to3dtensor(o):
    o = totensor(o)
    if o.ndim == 3: return o
    elif o.ndim == 1: return o[None, None]
    elif o.ndim == 2: return o[:, None]
    assert False, f'Please, review input dimensions {o.ndim}'


def to2dtensor(o):
    o = totensor(o)
    if o.ndim == 2: return o
    elif o.ndim == 1: return o[None]
    elif o.ndim == 3: return o[0]
    assert False, f'Please, review input dimensions {o.ndim}'


def to1dtensor(o):
    o = totensor(o)
    if o.ndim == 1: return o
    elif o.ndim == 3: return o[0,0]
    if o.ndim == 2: return o[0]
    assert False, f'Please, review input dimensions {o.ndim}'


def to3darray(o):
    o = toarray(o)
    if o.ndim == 3: return o
    elif o.ndim == 1: return o[None, None]
    elif o.ndim == 2: return o[:, None]
    assert False, f'Please, review input dimensions {o.ndim}'


def to2darray(o):
    o = toarray(o)
    if o.ndim == 2: return o
    elif o.ndim == 1: return o[None]
    elif o.ndim == 3: return o[0]
    assert False, f'Please, review input dimensions {o.ndim}'


def to1darray(o):
    o = toarray(o)
    if o.ndim == 1: return o
    elif o.ndim == 3: o = o[0,0]
    elif o.ndim == 2: o = o[0]
    assert False, f'Please, review input dimensions {o.ndim}'
    
    
def to3d(o):
    if o.ndim == 3: return o
    if isinstance(o, np.ndarray): return to3darray(o)
    if isinstance(o, torch.Tensor): return to3dtensor(o)
    
    
def to2d(o):
    if o.ndim == 2: return o
    if isinstance(o, np.ndarray): return to2darray(o)
    if isinstance(o, torch.Tensor): return to2dtensor(o)
    
    
def to1d(o):
    if o.ndim == 1: return o
    if isinstance(o, np.ndarray): return to1darray(o)
    if isinstance(o, torch.Tensor): return to1dtensor(o)
    
    
def to2dPlus(o):
    if o.ndim >= 2: return o
    if isinstance(o, np.ndarray): return to2darray(o)
    elif isinstance(o, torch.Tensor): return to2dtensor(o)
    
    
def to3dPlus(o):
    if o.ndim >= 3: return o
    if isinstance(o, np.ndarray): return to3darray(o)
    elif isinstance(o, torch.Tensor): return to3dtensor(o)
    
    
def to2dPlusTensor(o):
    return to2dPlus(totensor(o))


def to2dPlusArray(o):
    return to2dPlus(toarray(o))


def to3dPlusTensor(o):
    return to3dPlus(totensor(o))


def to3dPlusArray(o):
    return to3dPlus(toarray(o))


def Todtype(dtype):
    def _to_type(o, dtype=dtype):
        if o.dtype == dtype: return o
        elif isinstance(o, torch.Tensor): o = o.to(dtype=dtype)
        elif isinstance(o, np.ndarray): o = o.astype(dtype)
        return o
    return _to_type

In [ ]:
#export
#tsai.utils
#export
def itemify(*o, tup_id=None): 
    items = L(*o).zip()
    if tup_id is not None: return L([item[tup_id] for item in items])
    else: return items
    
def ifnoneelse(a, b, c=None):
    "`b` if `a` is None else `c`"
    return b if a is None else ifnone(c, a)

def cycle_dl(dl):
    for _ in dl: _

In [ ]:
#export
#fastcore.foundations
def _is_array(x): return hasattr(x,'__array__') or hasattr(x,'iloc')
def _listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str) or _is_array(o): return [o]
    if is_iter(o): return list(o)
    return [o]

In [ ]:
#export
#tsai.utils
def stack(o, axis=0):
    if isinstance(o[0], torch.Tensor): return torch.stack(tuple(o), dim=axis)
    else: return np.stack(o, axis)

#### Datasets

NumpyDatasets:
- types for X and y: _xtype, _ytype (with show method)
- itemify(X) and itemify(y), (X[i],), are returned since we zip only a single iterable, therefore selction for [0] needed
- items: (itemify(X), itemify(y))
- init tfms
- init tls from zip(items, tfms)
- init n_inp
- init types for transformed lists
- ptls:  type(ptl(it)), ptls: tensor versions of the transformed lists, e.g. normally just stacked items of the tls --> creating these ptls takes a long time, but it is only a one time expense


NumpyDatasets: items (X,y)
(fastai)Datasets: items[[X1,y1], ... [Xn,yn]]

Datasets is a FilteredBase subclass --> handles splits fastai v2 style, subsets, train, valid, ...

In [ ]:
#export
#tsai.data.core
class NumpyTensor(TensorBase):
    "Returns a `tensor` with subclass `NumpyTensor` that has a show method"
    def __new__(cls, o, **kwargs):
        if isinstance(o, (list, L)): o = stack(o)
        res = cast(tensor(o), cls)
        res._meta = kwargs
        return res
    def __getitem__(self, idx):
        res = super().__getitem__(idx)
        return res.as_subclass(type(self))
    def __repr__(self):
        if self.numel() == 1: return f'{self}'
        else: return f'NumpyTensor(shape:{list(self.shape)})'
    def show(self, ax=None, ctx=None, title=None, title_color='black', **kwargs):
        if self.ndim != 2: self = type(self)(to2dtensor(self))
        ax = ifnone(ax,ctx)
        if ax is None: fig, ax = plt.subplots(**kwargs)
        ax.plot(self.T)
        ax.axis(xmin=0, xmax=self.shape[-1] - 1)
        ax.set_title(title, weight='bold', color=title_color)
        plt.tight_layout()
        return ax

In [ ]:
#export
#tsai.data.core
class NumpyDatasets(Datasets):
    "A dataset that creates tuples from X (and y) and applies `tfms` of type item_tfms"
    _xtype, _ytype = NumpyTensor, None # Expected X and y output types (must have a show method)
    def __init__(self, X=None, y=None, items=None, tfms=None, tls=None, n_inp=None, dl_type=None, inplace=True, **kwargs):
        self.inplace = inplace
        if tls is None:
            X = itemify(X, tup_id=0)
            y = itemify(y, tup_id=0) if y is not None else y
            items = tuple((X,)) if y is None else tuple((X,y))
            self.tfms = L(ifnone(tfms,[None]*len(ifnone(tls,items))))
        self.tls = L(tls if tls else [TfmdLists(item, t, **kwargs) for item,t in zip(items,self.tfms)])
        self.n_inp = (1 if len(self.tls)==1 else len(self.tls)-1) if n_inp is None else n_inp
        if len(self.tls[0]) > 0:
            self.types = L([ifnone(_typ, type(tl[0]) if isinstance(tl[0], torch.Tensor) else tensor) 
                            for tl,_typ in zip(self.tls, [self._xtype, self._ytype])])
            self.ptls = L([tl if not self.inplace else tl[:] if type(tl[0]).__name__ == 'memmap' 
                           else tensor(stack(tl[:])) for tl in self.tls])

    def __getitem__(self, it):
        return tuple([typ(ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])

    def subset(self, i): return type(self)(tls=L(tl.subset(i) for tl in self.tls), n_inp=self.n_inp, inplace=self.inplace, tfms=self.tfms)

    def _new(self, X, *args, y=None, **kwargs):
        items = ifnoneelse(y,tuple((X,)),tuple((X, y)))
        return super()._new(items, tfms=self.tfms, do_setup=False, **kwargs)

    def show_at(self, idx, **kwargs):
        self.show(self[idx], **kwargs)
        plt.show()

    @property
    def items(self): return tuple([tl.items for tl in self.tls])
    @items.setter
    def items(self, vs):
        for tl,c in zip(self.tls, vs): tl.items = v

In [ ]:
#export
#tsai.data.core
#original
class TSDatasets(NumpyDatasets):
    "A dataset that creates tuples from X (and y) and applies `item_tfms`"
    _xtype, _ytype = TSTensor, None # Expected X and y output types (torch.Tensor - default - or subclass)
    def __init__(self, X=None, y=None, items=None, sel_vars=None, sel_steps=None, tfms=None, tls=None, n_inp=None, dl_type=None, 
                 inplace=True, **kwargs):
        self.inplace = inplace
        if tls is None: 
            X = itemify(X, tup_id=0)
            y = itemify(y, tup_id=0) if y is not None else y
            items = tuple((X)) if y is None else tuple((X,y))
            self.tfms = L(ifnone(tfms,[None]*len(ifnone(tls,items))))
        self.sel_vars = ifnone(sel_vars, slice(None))
        self.sel_steps = ifnone(sel_steps,slice(None))
        self.tls = L(tls if tls else [TfmdLists(item, t, **kwargs) for item,t in zip(items,self.tfms)])
        self.n_inp = (1 if len(self.tls)==1 else len(self.tls)-1) if n_inp is None else n_inp
        if len(self.tls[0]) > 0: 
            self.ptls = L([tl if not self.inplace else tl[:] if type(tl[0]).__name__ == 'memmap' else tensor(tl[:]) for tl in self.tls])
            self.types = [ifnone(_typ, type(tl[0]) if isinstance(tl[0], torch.Tensor) else tensor) for tl,_typ in zip(self.tls, [self._xtype, self._ytype])]
    
    def __getitem__(self, it):
        return tuple([typ(ptl[it])[...,self.sel_vars, self.sel_steps] if i==0 else typ(ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])
    
    def subset(self, i): return type(self)(tls=L(tl.subset(i) for tl in self.tls), n_inp=self.n_inp, 
                                           inplace=self.inplace, tfms=self.tfms, sel_vars=self.sel_vars, sel_steps=self.sel_steps)
    @property
    def vars(self): return self[0][0].shape[-2]
    @property
    def len(self): return self[0][0].shape[-1]

In [ ]:
#export
#tsai.data.core
## slightly adapted version
##NOTE TODO: Why does _ytype=TensorFloat not work (autograd fails)
class TSDatasets2(NumpyDatasets):
    "A dataset that creates tuples from X (and y) and applies `item_tfms`"
    _xtype, _xdistype, _ytype = TSTensor, TSIntTensor, None # Expected X and y output types (torch.Tensor - default - or subclass)
    def __init__(self, X=None, X_dis=None, y=None, items=None, sel_vars=None, sel_steps=None, tfms=None, tls=None, n_inp=None, dl_type=None,
                 inplace=True, **kwargs):
        self.inplace = inplace
        if tls is None:
            X = itemify(to3darray(X), tup_id=0)
            X_dis = itemify(to3darray(X_dis), tup_id=0) if X_dis is not None else X_dis
            y = itemify(y, tup_id=0) if y is not None else y
            items = tuple((X,)) if y is None else tuple((X,y))
            if X_dis is not None: items = tuple((X, X_dis, y)) if y is not None else tuple(X, X_dis,)
            self.tfms = L(ifnone(tfms,[None]*len(ifnone(tls,items))))
            
#         if X_dis is not None: self.X_dis = X_dis
       
        self.sel_vars = ifnone(sel_vars, slice(None))
        self.sel_steps = ifnone(sel_steps,slice(None))
        self.tls = L(tls if tls else [TfmdLists(item, t, **kwargs) for item,t in zip(items,self.tfms)])
        self.n_inp = (1 if len(self.tls)==1 else len(self.tls)-1) if n_inp is None else n_inp
        if len(self.tls[0]) > 0:
            _tls_types = [self._xtype, self._ytype] if len(self.tls)==2 else [self._xtype, self._xdistype, self._ytype]
            print(_tls_types)
#             print(len(self.tls))
#             for tl,_typ in zip(self.tls, _tls_types):
#                 print (len(tl), _typ, type(tl[0]), isinstance(tl[0], torch.Tensor))
            self.types = L([ifnone(_typ, type(tl[0]) if isinstance(tl[0], torch.Tensor) else tensor) for 
                            tl,_typ in zip(self.tls, _tls_types)])
            self.ptls = L([tl if not self.inplace else tl[:] if type(tl[0]).__name__ == 'memmap' else 
                           tensor(stack(tl[:])) for tl in self.tls])

    def __getitem__(self, it):
        
#         for i,(ptl,typ) in enumerate(zip(self.ptls,self.types)):
#             print (i, typ)
        
#         return tuple([typ(ptl[it])[...,self.sel_vars, self.sel_steps] if i==0 else 
#                       typ(ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])
        ## do not enable slicing for now 
        return tuple([typ(ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])
    

    def subset(self, i): return type(self)(tls=L(tl.subset(i) for tl in self.tls), n_inp=self.n_inp, 
                                           inplace=self.inplace, tfms=self.tfms,
                                           sel_vars=self.sel_vars, sel_steps=self.sel_steps)
    @property
    def vars(self): return self[0][0].shape[-2]
    @property
    def len(self): return self[0][0].shape[-1]


#### DataLoaders

fastaiv2 DataLoader:  
`def create_batch(self, b): return (fa_collate,fa_convert)[self.prebatched](b)`  
- collate collates several tensors together into one (batch)-tensor
- convert array->tensor 
`def create_item(self, s):  return next(self.it) if s is None else self.dataset[s]`


no need for collating and converting in the NumpyDataLoader


NumpyDataLoader:
- it is a subclass of TfmdDL, TfmdDL is a subclass of the standard v2 Dataloader:

The following parts are redefined:
- create_batch: slice instead of collate and convert (already collated and converted)
- create_item

NumpyDataLoader setup:
- for nm in _batch_tfms (after_item, before_batch, after_batch)
- after_item.setup(self) is called --> Pipelines, and Dataloader as parameter

`DataLoader.__iter__`:  
 - for b in _loaders[self.fake_l.num_workers==0](self.fake_l): just chooses the right loader (multiproc or not), casts self.fake_l to this loader, and then iterates over that  loader:
 - casts b to correct device
 - yields after_batch(b)
- after_batch in DataLoader is noop, it is a Pipeline of optional transforms


In [ ]:
#export
#tsai.data.core

class NumpyTensorBlock():
    def __init__(self, type_tfms=None, item_tfms=None, batch_tfms=None, dl_type=None, dls_kwargs=None):
        self.type_tfms  =                 L(type_tfms)
        self.item_tfms  = ToNumpyTensor + L(item_tfms)
        self.batch_tfms =                 L(batch_tfms)
        self.dl_type,self.dls_kwargs = dl_type,({} if dls_kwargs is None else dls_kwargs)
        
class TSTensorBlock():
    def __init__(self, type_tfms=None, item_tfms=None, batch_tfms=None, dl_type=None, dls_kwargs=None):
        self.type_tfms  =              L(type_tfms)
        self.item_tfms  = ToTSTensor + L(item_tfms)
        self.batch_tfms =              L(batch_tfms)
        self.dl_type,self.dls_kwargs = dl_type,({} if dls_kwargs is None else dls_kwargs)

In [ ]:
#export
#tsai.data.core
_batch_tfms = ('after_item','before_batch','after_batch')

class NumpyDataLoader(TfmdDL):
    idxs = None
    do_item = noops # create batch returns indices
    def __init__(self, dataset, bs=64, shuffle=False, num_workers=None, verbose=False, do_setup=True, batch_tfms=None, **kwargs):
        '''batch_tfms == after_batch (either can be used)'''
        if num_workers is None: num_workers = min(16, defaults.cpus)
        for nm in _batch_tfms: 
            if nm == 'after_batch' and batch_tfms is not None: kwargs[nm] = Pipeline(batch_tfms)
            else: kwargs[nm] = Pipeline(kwargs.get(nm,None))
        bs = min(bs, len(dataset))
        super().__init__(dataset, bs=bs, shuffle=shuffle, num_workers=num_workers, **kwargs)
        if do_setup:
            for nm in _batch_tfms:
                pv(f"Setting up {nm}: {kwargs[nm]}", verbose)
                kwargs[nm].setup(self)

    def create_batch(self, b):
        it = b if self.shuffle else slice(b[0], b[0] + self.bs)
        self.idxs = b
        return self.dataset[it]

    def create_item(self, s): return s

    def get_idxs(self):
        idxs = Inf.count if self.indexed else Inf.nones
        if self.n is not None: idxs = list(range(len(self.dataset)))
        if self.shuffle: idxs = self.shuffle_fn(idxs)
        return idxs

    @delegates(plt.subplots)
    def show_batch(self, b=None, ctxs=None, max_n=9, nrows=3, ncols=3, figsize=(16, 10), **kwargs):
        b = self.one_batch()
        db = self.decode_batch(b, max_n=max_n)
        if figsize is None: figsize = (ncols*6, max_n//ncols*4)
        if ctxs is None: ctxs = get_grid(min(len(db), nrows*ncols), nrows=None, ncols=ncols, figsize=figsize, **kwargs)
        for i,ctx in enumerate(ctxs): show_tuple(db[i], ctx=ctx)

    @delegates(plt.subplots)
    def show_results(self, b, preds, ctxs=None, max_n=9, nrows=3, ncols=3, figsize=(16, 10), **kwargs):
        t = self.decode_batch(b, max_n=max_n)
        p = self.decode_batch((b[0],preds), max_n=max_n)
        if figsize is None: figsize = (ncols*6, max_n//ncols*4)
        if ctxs is None: ctxs = get_grid(min(len(t), nrows*ncols), nrows=None, ncols=ncols, figsize=figsize, **kwargs)
        for i,ctx in enumerate(ctxs): 
            title = f'True: {t[i][1]}\nPred: {p[i][1]}'
            color = 'green' if t[i][1] == p[i][1] else 'red'
            t[i][0].show(ctx=ctx, title=title, title_color=color)

@delegates(plt.subplots)
def show_tuple(tup, **kwargs):
    "Display a timeseries plot from a decoded tuple"
    tup[0].show(title='unlabeled' if len(tup) == 1 else tup[1], **kwargs)
    
class TSDataLoader(NumpyDataLoader): 
    @property
    def vars(self): return self.dataset[0][0].shape[-2]
    @property
    def len(self): return self.dataset[0][0].shape[-1]

In [ ]:
#export
#tsai.data.core

_batch_tfms = ('after_item','before_batch','after_batch')

class NumpyDataLoaders(DataLoaders):
    _xblock = NumpyTensorBlock
    _dl_type = NumpyDataLoader 
    def __init__(self, *loaders, path='.', device=default_device()):
        self.loaders,self.path = list(loaders),Path(path)
        self.device = device
        
    @classmethod
    @delegates(DataLoaders.from_dblock)
    def from_numpy(cls, X, y=None, splitter=None, valid_pct=0.2, seed=0, item_tfms=None, batch_tfms=None, **kwargs):
        "Create timeseries dataloaders from arrays (X and y, unless unlabeled)"
        if splitter is None: splitter = RandomSplitter(valid_pct=valid_pct, seed=seed)
        getters = [ItemGetter(0), ItemGetter(1)] if y is not None else [ItemGetter(0)]
        dblock = DataBlock(blocks=(cls._xblock, CategoryBlock),
                           getters=getters,
                           splitter=splitter,
                           item_tfms=item_tfms,
                           batch_tfms=batch_tfms)

        source = itemify(X) if y is None else itemify(X,y)
        return cls.from_dblock(dblock, source, **kwargs)

    @classmethod
    def from_dsets(cls, *ds, path='.', bs=64, num_workers=0, batch_tfms=None, device=None, shuffle_train=True, **kwargs):
        default = (shuffle_train,) + (False,) * (len(ds)-1)
        defaults = {'shuffle': default, 'drop_last': default}
        kwargs = merge(defaults, {k: tuplify(v, match=ds) for k,v in kwargs.items()})
        kwargs = [{k: v[i] for k,v in kwargs.items()} for i in range_of(ds)]
        if not is_listy(bs): bs = [bs]
        if len(bs) != len(ds): bs = bs * len(ds)
        device = ifnone(device,default_device())
        return cls(*[cls._dl_type(d, bs=b, num_workers=num_workers, batch_tfms=batch_tfms, **k) \
                     for d,k,b in zip(ds, kwargs, bs)], path=path, device=device)

class TSDataLoaders(NumpyDataLoaders):
    _xblock = TSTensorBlock
    _dl_type = TSDataLoader

#### Transforms

In [ ]:
#export
#tsai.data.transform
class TSStandardize(Transform):
    "Standardize/destd batch of `NumpyTensor` or `TSTensor`"
    parameters, order = L('mean', 'std'), 99
    def __init__(self, mean=None, std=None, by_sample=False, by_var=False, verbose=False):
        self.mean = tensor(mean) if mean is not None else None
        self.std = tensor(std) if std is not None else None
        self.by_sample, self.by_var = by_sample, by_var
        if by_sample and by_var: self.axes = (2)
        elif by_sample: self.axes = (1, 2)
        elif by_var: self.axes = (0, 2)
        else: self.axes = ()
        self.verbose = verbose

    @classmethod
    def from_stats(cls, mean, std): return cls(mean, std)

    def setups(self, dl: DataLoader):
        if self.mean is None or self.std is None:
            pv(f'{self.__class__.__name__} setup mean={self.mean}, std={self.std}, by_sample={self.by_sample}, by_var={self.by_var}', self.verbose)
#             x, *_ = dl.one_batch() ##??
            x = dl.ptls[0] ## modification
            self.mean, self.std = x.mean(self.axes, keepdim=self.axes!=()), x.std(self.axes, keepdim=self.axes!=()) + 1e-7
            pv(f'mean: {self.mean}  std: {self.std}\n', self.verbose)

    def encodes(self, x:(NumpyTensor, TSTensor)): 
        pv('standardize encodes', self.verbose)
        if self.by_sample: self.mean, self.std = x.mean(self.axes, keepdim=self.axes!=()), x.std(self.axes, keepdim=self.axes!=()) + 1e-7
        return (x - self.mean) / self.std

In [ ]:
#export
@patch
def mul_min(x:(torch.Tensor, TSTensor, NumpyTensor), axes=(), keepdim=False):
    if axes == (): return retain_type(x.min(), x)
    axes = reversed(sorted(axes if is_listy(axes) else [axes]))
    min_x = x
    for ax in axes: min_x, _ = min_x.min(ax, keepdim)
    return retain_type(min_x, x)

@patch
def mul_max(x:(torch.Tensor, TSTensor, NumpyTensor), axes=(), keepdim=False):
    if axes == (): return retain_type(x.max(), x)
    axes = reversed(sorted(axes if is_listy(axes) else [axes]))
    max_x = x
    for ax in axes: max_x, _ = max_x.max(ax, keepdim)
    return retain_type(max_x, x)


class TSNormalize(Transform):
    "Normalize/denorm batch of `NumpyTensor` or `TSTensor`"
    parameters, order = L('min', 'max'), 99

    def __init__(self, min=None, max=None, range_min=-1, range_max=1, by_sample=True, by_var=False, verbose=False):
        self.min = tensor(min) if min is not None else None
        self.max = tensor(max) if max is not None else None
        self.range_min, self.range_max = range_min, range_max
        self.by_sample, self.by_var = by_sample, by_var
        if by_sample and by_var: self.axes = (2)
        elif by_sample: self.axes = (1, 2)
        elif by_var: self.axes = (0, 2)
        else: self.axes = ()
        self.verbose = verbose
        
    @classmethod
    def from_stats(cls, min, max, range_min=0, range_max=1): return cls(min, max, self.range_min, self.range_max)

    def setups(self, dl: DataLoader):
        if self.min is None or self.max is None:
            pv(f'{self.__class__.__name__} setup min={self.min}, max={self.max}, range_min={self.range_min}, range_max={self.range_max}, by_sample={self.by_sample}, by_var={self.by_var}',  self.verbose)
#             x, *_ = dl.one_batch()
            x = dl.ptls[0] 
            self.min, self.max = x.mul_min(self.axes, keepdim=self.axes!=()), x.mul_max(self.axes, keepdim=self.axes!=())
            pv(f'min: {self.min}  max: {self.max}\n', self.verbose)

    def encodes(self, x:(NumpyTensor, TSTensor)):
        if self.by_sample: self.min, self.max = x.mul_min(self.axes, keepdim=self.axes!=()), x.mul_max(self.axes, keepdim=self.axes!=())
        return ((x - self.min) / (self.max - self.min)) * (self.range_max - self.range_min) + self.range_min

### Get Data

In [ ]:
nrows = 200000
df_main = pd.read_csv('./data/custom/bi_sample_anon.csv', nrows=nrows)

In [ ]:
## simple config
x_cols = [[f'x{i}_{j}' for j in range(10)] for i in range(6)]
dep = 'y0'
n_train = 8000

items is a list of (x,y) tuples  
methods called when passing items to DataBlock:
- DataBlock.datasets(items, ....)
- Dataset(items, ...)
- TfmdList(items, ...)
- L(items, ...)
- CollBase.__init__(items, ...)



In [ ]:
## [(x_1, y_1), ..., (x_n, y_n)]
items = df_to_items(df_main, x_cols, dep, n_train)[0]

(200000, 6, 10)


In [ ]:
#export
def items_to_arrays(items):
    '''convert list of item tuples into X,y numpy arrays (for use with numpy dataloader)'''
#     return np.stack([x[0] for x in items]), np.stack([x[1] for x in items])
    return tuple(np.stack([x[i] for x in items]) for i in range(len(items[0])))

In [ ]:
X,y = items_to_arrays(items)

In [ ]:
X.shape, y.shape

((200000, 6, 10), (200000,))

In [ ]:
##use FixedSplitter
# splits = [L(range(160000)), L(range(160000,185000))]
splits = FixedSplitter()(df_main)
# splits = [L(range(1600)), L(range(1600,1850))]
tfms  = [None, None]

In [ ]:
%%time
nsets = NumpyDatasets(X[:100], y[:100])

CPU times: user 16.2 ms, sys: 30 µs, total: 16.2 ms
Wall time: 34.7 ms


In [ ]:
FixedSplitter??

In [ ]:
def TSSplitter(train_perc=0.8):
    def _inner(o, **kwargs):
        return L(range(0, int(len(o)*train_perc))), L(range(int(len(o)*train_perc), len(o)))
    return _inner

In [ ]:
splits=TSSplitter()(df_main.iloc[:100])

In [ ]:
dsets = TSDatasets2(X=X[:100], y=y[:100], tfms=tfms, splits=splits, inplace=True)

[<class 'torchtools.data.TSTensor'>, None]


In [ ]:
dsets.train[0]

[<class 'torchtools.data.TSTensor'>, None]


(TSTensor([[  -7.8431,  100.0000,   -6.4977,  -24.5098,  -33.2226,  -42.7350,
           -55.2486,  -56.4972,  -30.6748,  100.0000],
         [ 100.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
             0.0000,    0.0000,    0.0000,    0.0000],
         [   0.0000,    1.0000,   -1.0000,   -2.0000,   -1.0000,    0.0000,
             0.0000,    0.0000,   -2.0000,    1.0000],
         [ -17.3010,  100.0000,  -30.6748,  -54.9451, -227.2727,  -27.8552,
          -400.0000, -147.0588, -100.0000,  -94.3396],
         [   0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
             0.0000,    0.0000,    0.0000,    0.0000],
         [   0.0000,    1.0000,    0.0000,   -1.0000,   -1.0000,   -1.0000,
            -1.0000,    0.0000,   -1.0000,   -2.0000]]), tensor(-56.4972))

In [ ]:
# assert isinstance(dsets[0][-1], TensorFloat)
assert isinstance(dsets[0][0], TSTensor)

In [ ]:
# dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0)
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0, 
#                                after_batch=[TSNormalize(by_var=False)
                               batch_tfms=[TSStandardize(by_sample=False, by_var=True)])

[<class 'torchtools.data.TSTensor'>, None]
[<class 'torchtools.data.TSTensor'>, None]


In [ ]:
dsets.train

[<class 'torchtools.data.TSTensor'>, None]


(#80) [(TSTensor([[  -7.8431,  100.0000,   -6.4977,  -24.5098,  -33.2226,  -42.7350,
          -55.2486,  -56.4972,  -30.6748,  100.0000],
        [ 100.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000],
        [   0.0000,    1.0000,   -1.0000,   -2.0000,   -1.0000,    0.0000,
            0.0000,    0.0000,   -2.0000,    1.0000],
        [ -17.3010,  100.0000,  -30.6748,  -54.9451, -227.2727,  -27.8552,
         -400.0000, -147.0588, -100.0000,  -94.3396],
        [   0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000],
        [   0.0000,    1.0000,    0.0000,   -1.0000,   -1.0000,   -1.0000,
           -1.0000,    0.0000,   -1.0000,   -2.0000]]), tensor(-56.4972)),(TSTensor([[-161.2903,  -42.3729,  -96.1538,  100.0000,  100.0000,  100.0000,
          100.0000,  -20.4918,  100.0000,  100.0000],
        [   0.0000,    0.0000,    0.0000,    0.0000,    0.0000, 

In [ ]:
dsets.train.ptls[0].mean((0,2))

[<class 'torchtools.data.TSTensor'>, None]


tensor([-1.3731,  0.4565, -0.0150, -2.2484, -0.8475,  0.0150])

In [ ]:
dls.after_batch[0].mean

tensor([[[-1.3731],
         [ 0.4565],
         [-0.0150],
         [-2.2484],
         [-0.8475],
         [ 0.0150]]], device='cuda:0')

In [ ]:
xb,yb = dls[0].one_batch()

In [ ]:
type(xb), type(yb)

(torchtools.data.TSTensor, torch.Tensor)

In [ ]:
xb.shape, yb.shape

(torch.Size([80, 6, 10]), torch.Size([80]))

In [ ]:
# norm = TSNormalize(by_sample=False, by_var=True)
ss = TSStandardize(by_sample=False, by_var=True, verbose=True)
tszero = TimestepZero(verbose=True)

In [ ]:
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0, 
                              after_batch=Pipeline(noop),
                            batch_tfms=[ss, tszero])

[<class 'torchtools.data.TSTensor'>, None]
[<class 'torchtools.data.TSTensor'>, None]
TSStandardize setup mean=None, std=None, by_sample=False, by_var=True
mean: tensor([[[-1.3731],
         [ 0.4565],
         [-0.0150],
         [-2.2484],
         [-0.8475],
         [ 0.0150]]])  std: tensor([[[89.8309],
         [39.1756],
         [ 1.6403],
         [89.9025],
         [38.9999],
         [ 1.6418]]])



In [ ]:
model = InceptionTimeSgm(6,1)
loss_fn = get_loss_fn('leaky_loss', alpha=0.5)
learn_small = Learner(model=model, loss_func=loss_fn, dls=dls, metrics=unweighted_profit)

In [ ]:
learn_small.fit_one_cycle(5)

epoch,train_loss,valid_loss,unweighted_profit,time
0,1.852032,1.844110,-6.288322,00:00
1,1.830647,1.884596,-6.288322,00:00
2,-1.580215,1.964164,-6.288322,00:00
3,-4.794927,2.063437,-6.288322,00:00
4,-7.789885,2.132627,-5.603391,00:00


standardize encodes
timestepzero
standardize encodes
standardize encodes
timestepzero
standardize encodes
standardize encodes
timestepzero
standardize encodes
standardize encodes
timestepzero
standardize encodes
standardize encodes
timestepzero
standardize encodes


In [ ]:
preds = learn_small.get_preds(0)

standardize encodes
timestepzero


standardize encodes
timestepzero


In [ ]:
unweighted_profit(*preds)

tensor(-8.0380)

In [ ]:
loss_fn(*preds)

tensor(-0.1025, requires_grad=True)

In [ ]:
preds[0].shape

torch.Size([80, 1])

Test InceptionTimeVar

In [ ]:
model = InceptionTimeVar(6,1)
loss_fn = get_loss_fn('leaky_loss', alpha=0.5)
loss_fn = nll_leaky_loss
learn_small = Learner(model=model, loss_func=loss_fn, dls=dls, metrics=unweighted_profit)

In [ ]:
learn_small.fit_one_cycle(1)

standardize encodes
timestepzero


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
xb,yb = dls.one_batch()

In [ ]:
nll_leaky_loss??

In [ ]:
for xb,yb in dls[1]:
    print(unweighted_profit(learn_small.model(xb)[:,0], yb))

In [ ]:
learn_small.model(xb)[:,1]

In [ ]:
preds = learn_small.get_preds(1)

### Workflow

- load data frame
- create a configuration: x_cols, y_cols [n_train optional]
- create X,y from df and configuration
- create splits and transforms
- create dsets
- define batch tranforms
- create dlets from datasets
- define loss and model
- create Learner -> learn

In [ ]:
nrows=200000
df_main = pd.read_csv('./data/custom/bi_sample_anon.csv', nrows=nrows)

In [ ]:
## simple config
x_cols = [[f'x{i}_{j}' for j in range(10)] for i in range(6)]
dep = 'y0'
n_train = 8000

In [ ]:
items = df_to_items(df_main, x_cols, dep, n_train)[0]
X,y = items_to_arrays(items)

(200000, 6, 10)


In [ ]:
splits = FixedSplitter(n_train, n_train+5000)(df_main)
splits = FixedSplitter(120000, 160000)(df_main)

tfms = [None, None]

In [ ]:
L(splits).map(len)

(#2) [120000,40000]

In [ ]:
dsets = TSDatasets2(X=X, y=y, tfms=tfms, splits=splits, inplace=True)

[<class 'torchtools.data.TSTensor'>, None]


In [ ]:
#tsai.data.transform
class TSStandardize(Transform):
    "Standardize/destd batch of `NumpyTensor` or `TSTensor`"
    parameters, order = L('mean', 'std'), 99
    def __init__(self, mean=None, std=None, by_sample=False, by_var=False, verbose=False):
        self.mean = tensor(mean) if mean is not None else None
        self.std = tensor(std) if std is not None else None
        self.by_sample, self.by_var = by_sample, by_var
        if by_sample and by_var: self.axes = (2)
        elif by_sample: self.axes = (1, 2)
        elif by_var: self.axes = (0, 2)
        else: self.axes = ()
        self.verbose = verbose

    @classmethod
    def from_stats(cls, mean, std): return cls(mean, std)

    def setups(self, dl: DataLoader):
        if self.mean is None or self.std is None:
            pv(f'{self.__class__.__name__} setup mean={self.mean}, std={self.std}, by_sample={self.by_sample}, by_var={self.by_var}', self.verbose)
#             x, *_ = dl.one_batch() ##??
            x = dl.ptls[0] ## modification
            self.mean, self.std = x.mean(self.axes, keepdim=self.axes!=()), x.std(self.axes, keepdim=self.axes!=()) + 1e-7
            pv(f'mean: {self.mean}  std: {self.std}\n', self.verbose)

    def encodes(self, x:(NumpyTensor, TSTensor)): 
        pv('standardize encodes', self.verbose)
        if self.by_sample: self.mean, self.std = x.mean(self.axes, keepdim=self.axes!=()), x.std(self.axes, keepdim=self.axes!=()) + 1e-7
        return (x - self.mean) / self.std

In [ ]:
aug_tfms = all_noise_augs(magnitude=0.4, verbose=True)+all_erasing_augs(
    magnitude=0.4, verbose=True)+all_zoom_augs(magnitude=0.4, verbose=True)

In [ ]:
# norm = TSNormalize(by_sample=False, by_var=True)
ss = TSStandardize(by_sample=False, by_var=True, verbose=True)
norm = TSNormalize(by_sample=False, by_var=True, verbose=True)


augs = TimestepZero(verbose=True)
augs = RandAugment(N=3, magnitude=0.4, tfms=aug_tfms)


tfms [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , TimeNormal: (TSTensor,object) -> encodes , Dimout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , Cutout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , TimestepZero: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , Crop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , RandomCrop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , Maskout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , Zoomin: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , Zoomout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , RandZoom: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , RandTimesteps:

In [ ]:
# %%time
# dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0)
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0, 
#                                after_batch=[TSNormalize(by_var=False)
#                                after_batch=Pipeline(noop), ##??
                               batch_tfms=[ss, augs],
#                                batch_tfms=[norm, augs]
                               
#                                batch_tfms=[norm]
                              )

[<class 'torchtools.data.TSTensor'>, None]
[<class 'torchtools.data.TSTensor'>, None]
TSStandardize setup mean=None, std=None, by_sample=False, by_var=True
mean: tensor([[[-1.3464],
         [-1.4978],
         [ 0.0117],
         [-1.7581],
         [-1.9539],
         [-0.0117]]])  std: tensor([[[86.8401],
         [89.4383],
         [ 1.7140],
         [86.1442],
         [89.4364],
         [ 1.7140]]])



In [ ]:
xb, yb = dls.one_batch()

cutout
crop
dimout


In [ ]:
xb

TSTensor([[[ 0.0000,  0.0000,  0.0000,  ...,  1.0000,  0.3522,  1.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  1.0000,  0.0398,  0.5152],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.2879,  1.0000,  0.5833],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.1000,  0.2000, -0.2000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.6203,  0.4634,  0.5686],
         [ 0.0000,  0.0000,  0.0000,  ...,  1.0000,  0.1273,  0.1077],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.5714,  1.0000,  1.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.3000,  0.1000,  0.3000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.5641,  0.1111,  0.6000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0703,  0.0703,  0.5152],


In [ ]:
type(yb), type(xb)

(torch.Tensor, torchtools.data.TSTensor)

In [ ]:
set_seed(123)
model = InceptionTimeSgm(6,1)
# model = InceptionTimeSgmOld(6,1)
# model = InceptionTime(6,1)

# loss_fn = F.cross_entropy
loss_fn = get_loss_fn('leaky_loss', alpha=0.5)
# loss_fn = F.mse_loss
# loss_fn = nll_leaky_loss

In [ ]:
learn_small = Learner(dls, model=model, loss_func=loss_fn, metrics=unweighted_profit)

In [ ]:
learn = Learner(dls, model=model, loss_func=loss_fn, metrics=unweighted_profit)

In [ ]:
model(xb).shape

torch.Size([256, 1])

In [ ]:
lr = 1e-5
wd = 0.03
learn.fit_one_cycle(20, lr_max=lr, wd=wd)

epoch,train_loss,valid_loss,unweighted_profit,time
0,-0.224696,-0.544650,0.005216,00:34
1,-0.187594,-0.550788,-0.001034,00:33
2,-0.502434,-0.547495,0.006702,00:33
3,-0.444595,-0.573383,0.004622,00:34
4,-0.446209,-0.621524,0.000000,00:34
5,-0.419853,-0.611233,0.004622,00:33
6,-0.006825,-0.624103,0.003256,00:33
7,-0.668853,-0.622011,0.008327,00:34
8,-0.859300,-0.618821,0.007159,00:34
9,-0.350139,-0.639828,0.002500,00:34


timenormal torch.Size([256, 6, 10])
timestepzero
maskout
randomcrop
ywarp
yscale
dimout
randtimestep
zoomin
timenormal torch.Size([256, 6, 10])
randzoom
ynormal
cutout
zoomin
ywarp
timestepzero
zoomout
randtimestep
randzoom
timestepzero
maskout
timewarp
randzoom
maskout
zoomout
ywarp
timestepzero
timestepzero
ynormal
cutout
randomcrop
timestepzero
timewarp
timestepzero
zoomout
randzoom
timestepzero
dimout
yscale
crop
timewarp
yscale
timenormal torch.Size([256, 6, 10])
randzoom
randomcrop
yscale
ynormal
timestepzero
randzoom
randtimestep
dimout
maskout
randomcrop
zoomin
maskout
randtimestep
zoomout
timenormal torch.Size([256, 6, 10])
randzoom
crop
timenormal torch.Size([256, 6, 10])
zoomin
dimout
yscale
cutout
ywarp
zoomin
zoomout
timewarp
cutout
timenormal torch.Size([256, 6, 10])
crop
timewarp
maskout
zoomout
zoomin
ynormal
timewarp
maskout
ynormal
randzoom
zoomin
randzoom
crop
randzoom
timewarp
zoomin
cutout
timestepzero
ywarp
timestepzero
zoomout
ywarp
timewarp
cutout
ywarp
randzoom

timestepzero
timenormal torch.Size([256, 6, 10])
ynormal
ynormal
yscale
randzoom
dimout
timenormal torch.Size([256, 6, 10])
ywarp
crop
timestepzero
maskout
yscale
timenormal torch.Size([256, 6, 10])
randomcrop
cutout
zoomin
randtimestep
randzoom
maskout
zoomin
timenormal torch.Size([256, 6, 10])
randtimestep
cutout
ynormal
timewarp
yscale
yscale
maskout
zoomin
zoomin
dimout
zoomout
crop
cutout
timenormal torch.Size([256, 6, 10])
zoomin
maskout
ywarp
randomcrop
dimout
zoomout
zoomin
ynormal
timestepzero
ywarp
yscale
ynormal
timenormal torch.Size([256, 6, 10])
cutout
randzoom
ynormal
zoomin
timewarp
yscale
zoomin
randzoom
yscale
crop
maskout
cutout
crop
timenormal torch.Size([256, 6, 10])
randtimestep
timenormal torch.Size([256, 6, 10])
dimout
crop
maskout
timestepzero
timestepzero
ynormal
crop
timestepzero
crop
zoomout
maskout
timestepzero
timenormal torch.Size([256, 6, 10])
timestepzero
timenormal torch.Size([256, 6, 10])
dimout
randzoom
timewarp
zoomin
randomcrop
ywarp
zoomin
zoomout


timenormal torch.Size([256, 6, 10])
randomcrop
maskout
randtimestep
maskout
randzoom
ywarp
randomcrop
timestepzero
cutout
timestepzero
ynormal
cutout
dimout
zoomout
ywarp
zoomin
randtimestep
timenormal torch.Size([256, 6, 10])
dimout
ynormal
crop
yscale
cutout
cutout
yscale
zoomout
yscale
maskout
randomcrop
timenormal torch.Size([256, 6, 10])
randtimestep
crop
randtimestep
timestepzero
maskout
maskout
ywarp
randtimestep
timenormal torch.Size([256, 6, 10])
ynormal
timestepzero
timenormal torch.Size([256, 6, 10])
timewarp
timestepzero
randzoom
crop
maskout
ywarp
cutout
crop
maskout
randtimestep
ynormal
dimout
timenormal torch.Size([256, 6, 10])
crop
randzoom
ynormal
zoomin
timewarp
timenormal torch.Size([256, 6, 10])
crop
crop
timestepzero
randzoom
randtimestep
ywarp
timewarp
timewarp
ywarp
crop
dimout
cutout
timestepzero
timenormal torch.Size([256, 6, 10])
zoomin
ynormal
timenormal torch.Size([256, 6, 10])
timewarp
randomcrop
timenormal torch.Size([256, 6, 10])
randomcrop
cutout
crop
ti

maskout
ywarp
zoomout
dimout
randomcrop
cutout
crop
yscale
ywarp
cutout
timenormal torch.Size([256, 6, 10])
ywarp
timestepzero
yscale
randtimestep
ynormal
timenormal torch.Size([256, 6, 10])
dimout
randomcrop
dimout
randzoom
timewarp
timenormal torch.Size([256, 6, 10])
zoomout
dimout
crop
timenormal torch.Size([256, 6, 10])
dimout
maskout
cutout
ywarp
maskout
ynormal
ywarp
timestepzero
randtimestep
zoomout
randzoom
randtimestep
yscale
timewarp
maskout
randzoom
crop
maskout
randtimestep
cutout
zoomout
cutout
yscale
dimout
randzoom
maskout
randomcrop
cutout
ynormal
timestepzero
timenormal torch.Size([256, 6, 10])
ynormal
randomcrop
yscale
randzoom
randomcrop
timenormal torch.Size([256, 6, 10])
crop
randzoom
randomcrop
timestepzero
randtimestep
dimout
randzoom
zoomin
randomcrop
randzoom
timestepzero
maskout
ynormal
timestepzero
maskout
randzoom
dimout
randzoom
timewarp
timestepzero
yscale
maskout
ywarp
ywarp
timenormal torch.Size([256, 6, 10])
maskout
timestepzero
randzoom
maskout
zoomin


...

ynormal
yscale
timenormal torch.Size([256, 6, 10])
maskout
timewarp
dimout
timestepzero
timenormal torch.Size([256, 6, 10])
zoomout
ywarp
dimout
randzoom
ynormal
ywarp
timestepzero
zoomout
maskout
dimout
timenormal torch.Size([256, 6, 10])
timestepzero
dimout
dimout
ynormal
timewarp
randomcrop
ywarp
timenormal torch.Size([256, 6, 10])
dimout
maskout
timewarp
maskout
ywarp
ynormal
maskout
zoomout
yscale
ynormal
ywarp
zoomout
zoomin
timewarp
crop
randomcrop
zoomout
randtimestep
ywarp
ynormal
zoomout
yscale
randzoom
zoomin
timenormal torch.Size([256, 6, 10])
yscale
timestepzero
dimout
timewarp
maskout
timenormal torch.Size([256, 6, 10])
yscale
dimout
maskout
ywarp
randomcrop
timenormal torch.Size([256, 6, 10])
ynormal
timestepzero
randzoom
ywarp
crop
crop
zoomin
randtimestep
timenormal torch.Size([256, 6, 10])
ynormal
ywarp
zoomin
timestepzero
ywarp
randomcrop
ywarp
randzoom
randomcrop
timestepzero
randzoom
crop
randtimestep
timestepzero
timewarp
randomcrop
maskout
zoomin
ynormal
timewarp

cutout
timewarp
maskout
timewarp
ynormal
timestepzero
timenormal torch.Size([256, 6, 10])
zoomout
randzoom
timestepzero
zoomout
crop
ynormal
maskout
randomcrop
timenormal torch.Size([256, 6, 10])
ynormal
maskout
dimout
randtimestep
crop
ywarp
zoomin
zoomout
zoomout
randtimestep
timenormal torch.Size([256, 6, 10])
yscale
randtimestep
zoomin
ywarp
randzoom
timewarp
timewarp
maskout
randtimestep
crop
zoomin
maskout
maskout
timestepzero
dimout
randtimestep
maskout
randzoom
crop
ynormal
timenormal torch.Size([256, 6, 10])
timestepzero
zoomin
maskout
yscale
dimout
ywarp
timestepzero
randzoom
timewarp
timewarp
yscale
dimout
zoomout
timestepzero
dimout
zoomout
timewarp
cutout
yscale
ynormal
zoomout
randtimestep
cutout
zoomin
timestepzero
randzoom
ywarp
yscale
timenormal torch.Size([256, 6, 10])
zoomin
timestepzero
randtimestep
zoomout
randomcrop
ywarp
crop
timestepzero
dimout
randzoom
crop
zoomout
maskout
timestepzero
zoomout
timewarp
yscale
timewarp
crop
zoomin
ywarp
crop
timestepzero
zoomout

cutout
randtimestep
randomcrop
yscale
randtimestep
ywarp
ynormal
randzoom
cutout
timestepzero
maskout
timewarp
yscale
timewarp
randtimestep
timenormal torch.Size([256, 6, 10])
randzoom
randtimestep
dimout
timestepzero
zoomout
yscale
timenormal torch.Size([256, 6, 10])
randtimestep
randtimestep
randomcrop
timewarp
ywarp
randzoom
randtimestep
cutout
randomcrop
zoomin
dimout
timewarp
cutout
zoomin
timenormal torch.Size([256, 6, 10])
zoomout
yscale
crop
randzoom
ynormal
timenormal torch.Size([256, 6, 10])
yscale
randzoom
crop
cutout
randomcrop
timenormal torch.Size([256, 6, 10])
ynormal
crop
yscale
timenormal torch.Size([256, 6, 10])
timewarp
randomcrop
ywarp
maskout
zoomin
ynormal
crop
ynormal
dimout
dimout
timenormal torch.Size([256, 6, 10])
ywarp
randomcrop
zoomout
timenormal torch.Size([256, 6, 10])
timewarp
timestepzero
zoomin
timenormal torch.Size([256, 6, 10])
maskout
randomcrop
zoomin
randtimestep
randomcrop
randzoom
zoomout
ynormal
yscale
ywarp
zoomin
randtimestep
timestepzero
ran

timewarp
crop
ynormal
timenormal torch.Size([256, 6, 10])
randzoom
maskout
ynormal
crop
cutout
yscale
randzoom
zoomin
timewarp
randomcrop
timenormal torch.Size([256, 6, 10])
maskout
randzoom
randzoom
randtimestep
yscale
yscale
zoomout
timestepzero
randtimestep
timestepzero
zoomout
ywarp
cutout
timenormal torch.Size([256, 6, 10])
ynormal
timewarp
timenormal torch.Size([256, 6, 10])
cutout
timestepzero
maskout
timewarp
yscale
randtimestep
randtimestep
zoomin
yscale
yscale
randtimestep
zoomout
dimout
timenormal torch.Size([256, 6, 10])
zoomin
zoomout
maskout
randomcrop
timestepzero
randomcrop
zoomin
randtimestep
maskout
yscale
randomcrop
timenormal torch.Size([256, 6, 10])
dimout
zoomin
ywarp
timenormal torch.Size([256, 6, 10])
zoomin
randtimestep
timestepzero
maskout
randzoom
zoomout
yscale
timenormal torch.Size([256, 6, 10])
randzoom
crop
ynormal
zoomin
ynormal
crop
randomcrop
dimout
zoomin
crop
timestepzero
yscale
timenormal torch.Size([256, 6, 10])
crop
dimout
randomcrop
randtimestep


randomcrop
randtimestep
dimout
zoomin
dimout
cutout
zoomin
yscale
randomcrop
ynormal
yscale
cutout
ynormal
yscale
maskout
ywarp
crop
cutout
crop
dimout
timewarp
timestepzero
yscale
randzoom
ynormal
cutout
yscale
zoomin
timenormal torch.Size([256, 6, 10])
crop
ywarp
crop
ynormal
yscale
randzoom
timewarp
timewarp
randtimestep
zoomin
timestepzero
cutout
maskout
randzoom
zoomout
yscale
ynormal
timenormal torch.Size([256, 6, 10])
ywarp
zoomin
randtimestep
timestepzero
maskout
crop
timestepzero
crop
zoomin
dimout
zoomin
crop
timestepzero
randtimestep
randzoom
timestepzero
ynormal
randzoom
crop
yscale
randomcrop
cutout
ywarp
ynormal
cutout
timenormal torch.Size([256, 6, 10])
ynormal
timewarp
ywarp
timestepzero
zoomin
maskout
cutout
zoomin
timewarp
randzoom
randtimestep
randtimestep
zoomin
cutout
ynormal
crop
ywarp
zoomout
cutout
timestepzero
randomcrop
dimout
timewarp
ywarp
ynormal
randomcrop
crop
yscale
dimout
randomcrop
crop
dimout
ywarp
zoomout
timenormal torch.Size([256, 6, 10])
timestepz

In [ ]:
lr = 1e-5
wd = 0.03
learn.fit_one_cycle(10, lr_max=lr, wd=wd)

epoch,train_loss,valid_loss,unweighted_profit,time
0,-0.077557,-0.202007,0.002114,00:35
1,-0.127840,-0.300461,0.000000,00:34
2,-0.125123,-0.430187,0.000000,00:34
3,-0.294518,-0.450853,0.004082,00:34
4,-0.495527,-0.480928,0.006582,00:30
5,-0.175598,-0.508318,0.002122,00:28
6,-0.401401,-0.553612,0.004622,00:28
7,-0.161987,-0.566590,0.002122,00:35
8,-0.317239,-0.537305,0.005216,00:30
9,-0.257952,-0.489680,-0.017476,00:28


timestepzero
maskout
timenormal torch.Size([256, 6, 10])
zoomin
randtimestep
cutout
crop
randomcrop
yscale
yscale
crop
ywarp
timenormal torch.Size([256, 6, 10])
crop
dimout
timenormal torch.Size([256, 6, 10])
zoomout
yscale
randzoom
randomcrop
ywarp
timenormal torch.Size([256, 6, 10])
randomcrop
cutout
ywarp
crop
timewarp
ywarp
ynormal
yscale
timewarp
maskout
timenormal torch.Size([256, 6, 10])
randzoom
zoomout
timewarp
crop
zoomout
randzoom
randtimestep
ynormal
ywarp
timestepzero
randtimestep
maskout
crop
zoomout
maskout
randomcrop
randzoom
timestepzero
cutout
randzoom
ynormal
timestepzero
randzoom
yscale
timewarp
randtimestep
timenormal torch.Size([256, 6, 10])
randzoom
yscale
timenormal torch.Size([256, 6, 10])
randomcrop
cutout
crop
timestepzero
ywarp
cutout
yscale
randzoom
maskout
dimout
cutout
zoomout
randzoom
timenormal torch.Size([256, 6, 10])
timewarp
zoomin
randomcrop
ywarp
timewarp
zoomout
cutout
zoomin
randzoom
dimout
timestepzero
timewarp
timenormal torch.Size([256, 6, 10]

crop
timewarp
maskout
timewarp
randomcrop
randtimestep
dimout
ywarp
maskout
yscale
randzoom
maskout
crop
zoomin
randzoom
dimout
timenormal torch.Size([256, 6, 10])
randtimestep
ywarp
cutout
maskout
cutout
zoomout
ywarp
timestepzero
ywarp
dimout
crop
dimout
randomcrop
timenormal torch.Size([256, 6, 10])
ynormal
ywarp
ynormal
crop
randomcrop
randzoom
dimout
randtimestep
cutout
zoomin
yscale
randzoom
timenormal torch.Size([256, 6, 10])
timewarp
randomcrop
yscale
timenormal torch.Size([256, 6, 10])
maskout
timewarp
timestepzero
zoomout
cutout
ynormal
crop
yscale
randomcrop
timestepzero
randzoom
ynormal
zoomout
randzoom
cutout
maskout
dimout
zoomin
yscale
crop
randtimestep
zoomin
timestepzero
randzoom
yscale
timewarp
timestepzero
ywarp
timestepzero
cutout
dimout
randzoom
cutout
timenormal torch.Size([256, 6, 10])
dimout
randomcrop
timenormal torch.Size([256, 6, 10])
randomcrop
maskout
dimout
ynormal
zoomin
maskout
randomcrop
timestepzero
timestepzero
timenormal torch.Size([256, 6, 10])
rand

ywarp
dimout
timenormal torch.Size([256, 6, 10])
randomcrop
cutout
timenormal torch.Size([256, 6, 10])
crop
ywarp
timenormal torch.Size([256, 6, 10])
zoomout
timewarp
timenormal torch.Size([256, 6, 10])
ywarp
timestepzero
crop
timenormal torch.Size([256, 6, 10])
crop
maskout
zoomin
ynormal
timewarp
crop
timewarp
cutout
ywarp
yscale
cutout
ywarp
ynormal
timewarp
dimout
randzoom
maskout
maskout
yscale
randzoom
crop
timestepzero
dimout
ywarp
ynormal
zoomout
maskout
cutout
randzoom
zoomin
randzoom
dimout
maskout
zoomin
timestepzero
timestepzero
zoomout
timewarp
zoomin
ywarp
crop
zoomout
cutout
ywarp
dimout
timestepzero
randzoom
cutout
zoomin
ywarp
timewarp
zoomin
yscale
timestepzero
timewarp
randomcrop
crop
timewarp
dimout
timestepzero
zoomout
dimout
crop
timenormal torch.Size([256, 6, 10])
timewarp
timenormal torch.Size([256, 6, 10])
dimout
randzoom
randomcrop
ywarp
timestepzero
yscale
ywarp
dimout
timestepzero
randtimestep
ywarp
ywarp
maskout
timenormal torch.Size([256, 6, 10])
timestepz

cutout
randtimestep
timestepzero
maskout
timestepzero
crop
dimout
timestepzero
zoomin
timewarp
zoomout
crop
ywarp
randzoom
maskout
ynormal
timewarp
dimout
crop
cutout
randomcrop
ynormal
crop
ywarp
ywarp
maskout
crop
zoomin
ywarp
randzoom
maskout
randtimestep
crop
timewarp
yscale
randomcrop
cutout
timewarp
timestepzero
ynormal
yscale
timestepzero
maskout
zoomout
timewarp
zoomin
ynormal
randtimestep
zoomout
randtimestep
dimout
randzoom
timenormal torch.Size([256, 6, 10])
ywarp
dimout
ywarp
randtimestep
ynormal
randomcrop
ywarp
ynormal
randomcrop
timestepzero
randzoom
timewarp
crop
timewarp
ynormal
crop
maskout
cutout
randomcrop
dimout
ywarp
maskout
maskout
randzoom
ywarp
randzoom
ynormal
timewarp
randtimestep
yscale
randzoom
timestepzero
ynormal
crop
zoomout
cutout
randomcrop
dimout
ywarp
zoomout
timestepzero
randomcrop
dimout
ywarp
randomcrop
zoomout
randzoom
timewarp
randomcrop
maskout
zoomout
ywarp
crop
timestepzero
randtimestep
timenormal torch.Size([256, 6, 10])
randomcrop
timestepz

...

timenormal torch.Size([256, 6, 10])
randtimestep
zoomout
zoomin
ynormal
yscale
zoomout
cutout
randzoom
maskout
cutout
timenormal torch.Size([256, 6, 10])
crop
cutout
dimout
cutout
maskout
zoomout
dimout
yscale
crop
randzoom
randtimestep
dimout
ynormal
randomcrop
zoomout
zoomout
crop
dimout
timestepzero
zoomout
yscale
randzoom
maskout
randomcrop
cutout
zoomin
randomcrop
timestepzero
timenormal torch.Size([256, 6, 10])
timewarp
maskout
zoomin
zoomout
timestepzero
maskout
randtimestep
zoomin
randtimestep
yscale
randtimestep
randomcrop
zoomin
dimout
ywarp
timenormal torch.Size([256, 6, 10])
zoomout
randzoom
randomcrop
maskout
timestepzero
cutout
zoomin
maskout
cutout
yscale
randtimestep
crop
maskout
timestepzero
timenormal torch.Size([256, 6, 10])
randomcrop
randtimestep
randzoom
zoomout
zoomin
cutout
timewarp
randomcrop
timestepzero
timewarp
timestepzero
randzoom
randzoom
timestepzero
timewarp
timewarp
randomcrop
timenormal torch.Size([256, 6, 10])
zoomout
maskout
ywarp
randomcrop
timenor

timestepzero
randtimestep
timewarp
zoomout
zoomin
timenormal torch.Size([256, 6, 10])
crop
ynormal
maskout
dimout
maskout
ywarp
zoomout
yscale
randzoom
timewarp
dimout
zoomin
ynormal
randomcrop
crop
dimout
crop
ywarp
cutout
yscale
maskout
crop
timestepzero
zoomin
timewarp
ynormal
randzoom
timewarp
crop
zoomout
timenormal torch.Size([256, 6, 10])
maskout
zoomout
randomcrop
ywarp
crop
crop
dimout
zoomout
zoomin
zoomout
ynormal
zoomin
randtimestep
timenormal torch.Size([256, 6, 10])
crop
dimout
randzoom
randtimestep
zoomin
timestepzero
crop
dimout
zoomout
crop
timewarp
cutout
zoomout
cutout
randomcrop
zoomout
zoomin
ynormal
randomcrop
zoomout
timestepzero
zoomout
timestepzero
ywarp
zoomout
ywarp
randtimestep
cutout
timestepzero
yscale
crop
randtimestep
timestepzero
yscale
dimout
ywarp
crop
timestepzero
timenormal torch.Size([256, 6, 10])
timestepzero
randtimestep
randomcrop
dimout
timenormal torch.Size([256, 6, 10])
crop
maskout
randtimestep
crop
yscale
randomcrop
timenormal torch.Size([2

maskout
randtimestep
zoomin
zoomout
ywarp
yscale
zoomout
randomcrop
dimout
ywarp
timenormal torch.Size([256, 6, 10])
dimout
maskout
crop
ywarp
dimout
crop
ynormal
cutout
yscale
randomcrop
randtimestep
ynormal
dimout
randzoom
zoomout
timenormal torch.Size([256, 6, 10])
zoomin
dimout
zoomout
zoomin
ywarp
timewarp
dimout
zoomout
randomcrop
timestepzero
randzoom
randtimestep
ywarp
ynormal
crop
timewarp
maskout
randtimestep
timestepzero
randzoom
randomcrop
timewarp
randzoom
randtimestep
zoomin
randtimestep
maskout
yscale
randtimestep
crop
cutout
zoomout
zoomin
randomcrop
timenormal torch.Size([256, 6, 10])
timewarp
timenormal torch.Size([256, 6, 10])
randzoom
zoomout
timewarp
timenormal torch.Size([256, 6, 10])
timestepzero
randtimestep
timestepzero
yscale
dimout
randzoom
maskout
randtimestep
timewarp
cutout
randomcrop
zoomout
timenormal torch.Size([256, 6, 10])
ywarp
crop
dimout
zoomin
timenormal torch.Size([256, 6, 10])
maskout
timewarp
zoomin
randzoom
zoomin
randtimestep
randomcrop
dimou

crop
zoomin
cutout
cutout
randzoom
maskout
randtimestep
timestepzero
yscale
timewarp
cutout
maskout
ynormal
randomcrop
zoomin
randzoom
dimout
zoomout
timewarp
randomcrop
crop
timenormal torch.Size([256, 6, 10])
randtimestep
yscale
ynormal
zoomout
crop
randzoom
ynormal
timewarp
maskout
randtimestep
yscale
timenormal torch.Size([256, 6, 10])
timewarp
maskout
maskout
ywarp
ynormal
zoomin
maskout
yscale
cutout
ynormal
timenormal torch.Size([256, 6, 10])
randzoom
timewarp
ynormal
randomcrop
timestepzero
timewarp
zoomout
crop
maskout
crop
randomcrop
randzoom
randtimestep
yscale
randomcrop
timewarp
maskout
ywarp
dimout
maskout
ynormal
ynormal
timewarp
zoomout
crop
timewarp
yscale
ywarp
dimout
ynormal
yscale
dimout
cutout
timewarp
randtimestep
randzoom
zoomin
ynormal
zoomout
timestepzero
yscale
cutout
maskout
zoomin
crop
randomcrop
dimout
ywarp
maskout
timestepzero
yscale
timewarp
maskout
randzoom
timenormal torch.Size([256, 6, 10])
maskout
randtimestep
timewarp
timestepzero
ynormal
dimout
zoo

randzoom
zoomin
timewarp
zoomout
randomcrop
timestepzero
dimout
maskout
timenormal torch.Size([256, 6, 10])
randtimestep
timewarp
zoomin
timewarp
maskout
randzoom
randomcrop
cutout
randtimestep
yscale
maskout
timenormal torch.Size([256, 6, 10])
cutout
ynormal
ywarp
ynormal
timewarp
cutout
dimout
cutout
zoomout
randzoom
ywarp
yscale
crop
ywarp
zoomin
ynormal
zoomin
randtimestep
ynormal
zoomout
zoomin
zoomin
ynormal
timestepzero
randtimestep
maskout
zoomin
timenormal torch.Size([256, 6, 10])
zoomin
ywarp
yscale
timenormal torch.Size([256, 6, 10])
maskout
timestepzero
zoomin
dimout
randomcrop
randtimestep
zoomin
maskout
yscale
randomcrop
randtimestep
timewarp
maskout
zoomout
timewarp
dimout
randzoom
timenormal torch.Size([256, 6, 10])
crop
yscale
timenormal torch.Size([256, 6, 10])
randomcrop
timewarp
cutout
yscale
timestepzero
cutout
yscale
randomcrop
cutout
zoomin
cutout
randzoom
timenormal torch.Size([256, 6, 10])
randtimestep
ynormal
randomcrop
randzoom
yscale
cutout
timestepzero
mask

In [ ]:
lr = 1e-5
wd = 0.03
learn.fit_one_cycle(10, lr_max=lr, wd=wd)

epoch,train_loss,valid_loss,unweighted_profit,time
0,-0.107629,-0.151523,0.012688,00:35
1,-0.040655,-0.193177,0.016546,00:36
2,-0.249821,-0.261672,-0.006601,00:39
3,-0.450666,-0.298814,-0.054354,00:40
4,-0.423675,-0.345513,-0.069043,00:37
5,-0.371526,-0.340570,-0.026569,00:35
6,-0.372692,-0.379481,-0.048209,00:36
7,-0.302936,-0.392738,-0.032105,00:35
8,-0.260164,-0.377967,-0.024336,00:36
9,-0.345824,-0.359013,-0.012835,00:40


standardize encodes
timestepzero
maskout
timenormal torch.Size([256, 6, 10])
standardize encodes
zoomin
randtimestep
cutout
standardize encodes
crop
randomcrop
yscale
standardize encodes
yscale
crop
ywarp
standardize encodes
timenormal torch.Size([256, 6, 10])
crop
dimout
standardize encodes
timenormal torch.Size([256, 6, 10])
zoomout
yscale
standardize encodes
randzoom
randomcrop
ywarp
standardize encodes
timenormal torch.Size([256, 6, 10])
randomcrop
cutout
standardize encodes
ywarp
crop
timewarp
standardize encodes
ywarp
ynormal
yscale
standardize encodes
timewarp
maskout
timenormal torch.Size([256, 6, 10])
standardize encodes
randzoom
zoomout
timewarp
standardize encodes
crop
zoomout
randzoom
standardize encodes
randtimestep
ynormal
ywarp
standardize encodes
timestepzero
randtimestep
maskout
standardize encodes
crop
zoomout
maskout
standardize encodes
randomcrop
randzoom
timestepzero
standardize encodes
cutout
randzoom
ynormal
standardize encodes
timestepzero
randzoom
yscale
standa

standardize encodes
zoomout
crop
zoomin
standardize encodes
dimout
crop
ywarp
standardize encodes
randomcrop
crop
timenormal torch.Size([256, 6, 10])
standardize encodes
maskout
timewarp
randtimestep
standardize encodes
cutout
timestepzero
timenormal torch.Size([256, 6, 10])
standardize encodes
yscale
timenormal torch.Size([256, 6, 10])
ywarp
standardize encodes
dimout
zoomout
timestepzero
standardize encodes
zoomin
randtimestep
randzoom
standardize encodes
yscale
maskout
randtimestep
standardize encodes
zoomin
dimout
timestepzero
standardize encodes
timestepzero
maskout
randomcrop
standardize encodes
maskout
ynormal
cutout
standardize encodes
yscale
zoomin
randtimestep
standardize encodes
cutout
dimout
zoomin
standardize encodes
randtimestep
timenormal torch.Size([256, 6, 10])
zoomin
standardize encodes
timewarp
ywarp
maskout
standardize encodes
timestepzero
timewarp
yscale
standardize encodes
timenormal torch.Size([256, 6, 10])
timewarp
randzoom
standardize encodes
crop
ywarp
zoomout

standardize encodes
zoomin
ywarp
zoomout
standardize encodes
dimout
ywarp
timenormal torch.Size([256, 6, 10])
standardize encodes
ywarp
yscale
cutout
standardize encodes
timenormal torch.Size([256, 6, 10])
crop
zoomout
standardize encodes
randzoom
ynormal
crop
standardize encodes
yscale
randtimestep
cutout
standardize encodes
cutout
zoomout
timenormal torch.Size([256, 6, 10])
standardize encodes
zoomout
yscale
randomcrop
standardize encodes
randomcrop
randzoom
timenormal torch.Size([256, 6, 10])
standardize encodes
randomcrop
maskout
zoomin
standardize encodes
cutout
crop
randomcrop
standardize encodes
randomcrop
crop
cutout
standardize encodes
yscale
zoomout
timenormal torch.Size([256, 6, 10])
standardize encodes
ynormal
timestepzero
crop
standardize encodes
timewarp
dimout
timestepzero
standardize encodes
randzoom
cutout
ynormal
standardize encodes
randtimestep
cutout
timenormal torch.Size([256, 6, 10])
standardize encodes
ynormal
cutout
randomcrop
standardize encodes
ywarp
zoomin
cu

standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes
standardize encodes


...

standardize encodes
randzoom
timewarp
maskout
standardize encodes
cutout
yscale
crop
standardize encodes
randomcrop
zoomin
zoomout
standardize encodes
timestepzero
timenormal torch.Size([256, 6, 10])
randtimestep
standardize encodes
timewarp
yscale
dimout
standardize encodes
yscale
timestepzero
zoomout
standardize encodes
randtimestep
randomcrop
crop
standardize encodes
cutout
randomcrop
timenormal torch.Size([256, 6, 10])
standardize encodes
randzoom
yscale
ynormal
standardize encodes
maskout
dimout
yscale
standardize encodes
crop
timewarp
yscale
standardize encodes
timewarp
cutout
dimout
standardize encodes
randzoom
ynormal
maskout
standardize encodes
timenormal torch.Size([256, 6, 10])
maskout
zoomin
standardize encodes
ywarp
randzoom
timewarp
standardize encodes
randomcrop
yscale
randzoom
standardize encodes
dimout
timenormal torch.Size([256, 6, 10])
timestepzero
standardize encodes
zoomout
zoomin
dimout
standardize encodes
zoomout
randzoom
yscale
standardize encodes
maskout
zoomou

standardize encodes
yscale
dimout
timenormal torch.Size([256, 6, 10])
standardize encodes
yscale
maskout
timenormal torch.Size([256, 6, 10])
standardize encodes
crop
randzoom
yscale
standardize encodes
yscale
cutout
ynormal
standardize encodes
ywarp
zoomout
timewarp
standardize encodes
timenormal torch.Size([256, 6, 10])
timewarp
cutout
standardize encodes
ynormal
timenormal torch.Size([256, 6, 10])
maskout
standardize encodes
crop
zoomout
randzoom
standardize encodes
ynormal
maskout
dimout
standardize encodes
timestepzero
timewarp
dimout
standardize encodes
crop
zoomout
randomcrop
standardize encodes
zoomin
ywarp
maskout
standardize encodes
maskout
randomcrop
ynormal
standardize encodes
randzoom
zoomout
crop
standardize encodes
timewarp
ywarp
timestepzero
standardize encodes
timewarp
timestepzero
ywarp
standardize encodes
randtimestep
maskout
timenormal torch.Size([256, 6, 10])
standardize encodes
zoomout
ynormal
cutout
standardize encodes
maskout
randzoom
randtimestep
standardize enc

standardize encodes
ywarp
zoomout
timestepzero
standardize encodes
zoomin
ywarp
maskout
standardize encodes
zoomout
crop
zoomin
standardize encodes
timewarp
zoomout
zoomin
standardize encodes
cutout
randzoom
ynormal
standardize encodes
ynormal
zoomout
randtimestep
standardize encodes
dimout
zoomin
zoomout
standardize encodes
ywarp
randtimestep
timenormal torch.Size([256, 6, 10])
standardize encodes
maskout
cutout
yscale
standardize encodes
maskout
timestepzero
randtimestep
standardize encodes
randtimestep
timestepzero
yscale
standardize encodes
yscale
zoomin
timewarp
standardize encodes
maskout
ynormal
randtimestep
standardize encodes
zoomin
crop
dimout
standardize encodes
dimout
ynormal
randzoom
standardize encodes
randomcrop
timewarp
randtimestep
standardize encodes
crop
zoomin
cutout
standardize encodes
cutout
randzoom
maskout
standardize encodes
randtimestep
timestepzero
yscale
standardize encodes
timewarp
cutout
maskout
standardize encodes
ynormal
randomcrop
zoomin
standardize enc

standardize encodes
randzoom
ynormal
ywarp
standardize encodes
maskout
timewarp
randomcrop
standardize encodes
timewarp
randomcrop
yscale
standardize encodes
randomcrop
timestepzero
maskout
standardize encodes
randomcrop
dimout
randzoom
standardize encodes
maskout
yscale
randzoom
standardize encodes
ywarp
zoomin
yscale
standardize encodes
timestepzero
ywarp
timewarp
standardize encodes
yscale
crop
ynormal
standardize encodes
cutout
randzoom
zoomin
standardize encodes
timewarp
crop
randtimestep
standardize encodes
timenormal torch.Size([256, 6, 10])
randzoom
timewarp
standardize encodes
cutout
timewarp
timestepzero
standardize encodes
timenormal torch.Size([256, 6, 10])
timestepzero
crop
standardize encodes
maskout
randzoom
crop
standardize encodes
timestepzero
timewarp
randomcrop
standardize encodes
crop
ywarp
yscale
standardize encodes
cutout
randomcrop
zoomin
standardize encodes
timewarp
zoomout
dimout
standardize encodes
timestepzero
zoomout
randtimestep
standardize encodes
timewarp

standardize encodes
yscale
randtimestep
ywarp
standardize encodes
yscale
cutout
zoomout
standardize encodes
timestepzero
randomcrop
cutout
standardize encodes
dimout
randzoom
randomcrop
standardize encodes
zoomin
maskout
randomcrop
standardize encodes
cutout
timenormal torch.Size([256, 6, 10])
dimout
standardize encodes
randtimestep
maskout
randzoom
standardize encodes
crop
randtimestep
cutout
standardize encodes
cutout
randomcrop
zoomout
standardize encodes
timestepzero
crop
ynormal
standardize encodes
randtimestep
maskout
timenormal torch.Size([256, 6, 10])
standardize encodes
maskout
ynormal
zoomin
standardize encodes
cutout
ywarp
timewarp
standardize encodes
randtimestep
maskout
dimout
standardize encodes
zoomin
dimout
zoomout
standardize encodes
ywarp
zoomin
crop
standardize encodes
timewarp
timestepzero
cutout
standardize encodes
zoomin
timewarp
ynormal
standardize encodes
randomcrop
zoomin
timenormal torch.Size([256, 6, 10])
standardize encodes
timenormal torch.Size([256, 6, 10]

In [ ]:
lr = 1e-5
wd = 0.03
learn.fit_one_cycle(10, lr_max=lr, wd=wd)

epoch,train_loss,valid_loss,unweighted_profit,time
0,-0.146464,-0.112648,-0.108095,00:23
1,-0.125785,-0.161719,0.035120,00:22
2,-0.363113,-0.250997,0.112466,00:22
3,-0.559274,-0.306511,-0.078894,00:22
4,-0.720092,-0.350686,0.025928,00:21
5,-0.905205,-0.372047,0.091667,00:22
6,-0.940773,-0.389973,0.043651,00:22
7,-1.175533,-0.396730,-0.044709,00:22
8,-0.942312,-0.403905,-0.002367,00:22
9,-1.023425,-0.401614,-0.024770,00:22


In [ ]:
preds = learn.get_preds()

In [ ]:
(preds[0]>-.85).sum()

tensor(3785)

In [ ]:
preds[0].min()

tensor(-0.9986)

In [ ]:
unweighted_profit(*preds, -.85)

tensor(0.1169)

In [ ]:
preds[0]

tensor([[-0.9809],
        [-0.9631],
        [-0.9636],
        ...,
        [-0.9863],
        [-0.9263],
        [-0.9740]])

In [ ]:
learn.dls.after_batch[1].tfms[2].verbose

True

In [ ]:
xb,yb = dls.one_batch()

In [ ]:
xb

tensor([[[ 1.0000e+02,  1.0000e+02, -9.0909e+01,  ...,  1.0000e+02,
           1.0000e+02, -4.5045e+01],
         [ 1.0000e+02,  1.0000e+02, -1.2500e+02,  ...,  1.0000e+02,
           1.0000e+02, -1.1364e+02],
         [ 2.0000e+00,  1.0000e+00, -1.0000e+00,  ...,  1.0000e+00,
           3.0000e+00, -1.0000e+00],
         [-1.0526e+02,  1.0000e+02, -1.2658e+02,  ...,  1.0000e+02,
          -4.2735e+01, -7.8740e+01],
         [-1.0417e+02,  1.0000e+02, -1.2658e+02,  ...,  1.0000e+02,
          -1.0753e+02, -5.3191e+01],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          -1.0000e+00,  0.0000e+00]],

        [[ 1.0000e+02, -7.0423e+01,  1.0000e+02,  ...,  1.0000e+02,
           1.0000e+02, -6.7114e+01],
         [ 1.0000e+02, -8.2645e+01,  1.0000e+02,  ...,  1.0000e+02,
           1.0000e+02, -8.8496e+01],
         [ 3.0000e+00, -2.0000e+00,  2.0000e+00,  ...,  3.0000e+00,
           2.0000e+00, -1.0000e+00],
         [-4.2194e+01,  1.0000e+02, -4.9751e+01,  ...

In [ ]:
unweighted_profit(*preds, threshold=0.35)

tensor(0.1437)

In [ ]:
lr = 1e-3
wd = 0.03
learn_small.fit_one_cycle(5, lr_max=lr)

/home/johannes/coding/python/fastai2/fastai2/learner.py:161: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  self.loss = self.loss_func(self.pred, *self.yb); self('after_loss')


KeyboardInterrupt: 

In [ ]:
lr = 1e-3
wd = 0.03
learn_small.fit_one_cycle(5, lr_max=lr)

epoch,train_loss,valid_loss,unweighted_profit,time
0,-1.916526,1.188686,-1.629233,00:34
1,-1.391512,0.284585,-0.771737,00:34
2,-0.395118,0.097377,-0.198835,00:34
3,-2.968876,0.317685,-0.801690,00:34
4,-2.926257,0.586281,-1.063769,00:34


In [ ]:
lr = 1e-3
wd = 0.03
learn_small.fit_one_cycle(5, lr_max=lr, wd=wd, div_final=100)

epoch,train_loss,valid_loss,time
0,-0.236150,0.046494,00:09
1,-0.223928,0.054178,00:09
2,-0.021284,0.062445,00:09
3,0.228936,0.036816,00:09
4,-0.043758,0.057718,00:09


In [ ]:
learn_small.model(xb)

tensor([[-0.1351],
        [-0.1443],
        [-0.1264],
        [-0.0213],
        [-0.0505],
        [-0.3119],
        [-0.1161],
        [-0.0085],
        [-0.0799],
        [ 0.0519],
        [-0.1666],
        [-0.1083],
        [-0.1038],
        [ 0.0175],
        [-0.1023],
        [-0.0011]], device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
opt = learn_small.opt

In [ ]:
opt.param_lists[0][0][0].shape

torch.Size([6, 39])

In [ ]:
lr = 1e-5
wd = 0.03
learn_small.fit_one_cycle(5, lr_max=lr, wd=0.03)

KeyboardInterrupt: 

In [ ]:
preds = learn_small.get_preds()

In [ ]:
(preds[0]>0).sum(), preds[0].shape[0]

(tensor(22105), 40000)

In [ ]:
InceptionTimeSgm??

In [ ]:
preds[0].squeeze().max()

tensor(0.4587)

In [ ]:
preds[0].std()

tensor(0.0719)

In [ ]:
tensor(1754)/tensor(5000.)

tensor(0.3508)

In [ ]:
2000/5000

0.4

In [ ]:
preds[0].shape[0]

5000

#### Modelling with RandAugment

In [ ]:
tfms = all_erasing_augs(magnitude=0.4) + all_noise_augs(magnitude=0.4) + all_zoom_augs(magnitude=0.4)

In [ ]:
splits = [L(range(160000)), L(range(160000,185000))]
# splits = [L(range(1600)), L(range(1600,1850))]

# tfms  = [None, None]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits, inplace=True)

# dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0)
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 256], num_workers=0, 
#                                after_batch=[TSNormalize(by_var=False)
                               batch_tfms=[TSNormalize(by_sample=False, by_var=True), ToTT, 
                                           RandAugment(N=3, magnitude=0.4, tfms=tfms)
                                           ])

tfms [Dimout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , Cutout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , TimestepZero: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , Crop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , RandomCrop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , Maskout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , TimeNormal: (TSTensor,object) -> encodes , Zoomin: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , Zoomout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , RandZoom: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , RandTimesteps:

In [ ]:
xb, yb = dls[0].one_batch()

ToTT


In [ ]:
dls.after_batch.tfms[3].magnitude

0.4

In [ ]:
# dls.show_batch()

In [ ]:
set_seed(1234)

In [ ]:
## for tsai
model = InceptionTimeSgm(6,1)
learn = Learner(dls, model, loss_fn, metrics=[unweighted_profit, partial(unweighted_profit, threshold=0.5)])

In [ ]:
learn.fit_one_cycle(2, lr_max=1e-5, wd=0.03)

epoch,train_loss,valid_loss,unweighted_profit,unweighted_profit,time
0,-0.088209,-0.120086,0.003209,0.000000,00:40
1,-0.284319,-0.147903,0.032609,0.000000,00:36


ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT


In [ ]:
preds = learn.get_preds()

In [ ]:
(preds[0]>-0.5).sum()

tensor(23892)

In [ ]:
unweighted_profit(*preds, threshold=0)

tensor(0.0326)

### Discrete

oguiza TSDatasets, possible modification
- allow to directly use torchtools tensor types
- allow more than one X

- toarray, to3darray, totensor in tsai.utils useful for handling cpu, cuda, 2d, 3d cases, also for augmentations
- to3darray: a[:, None] assumes that the 2d array has batch, sequence dimensions, i.e. channel dimension missing

In [ ]:
#int
x_cont, x_dis, dep, n_train = get_discrete_config()

In [ ]:
#int
items_d, _ = df_to_items_discrete(df_main, [x_cont, x_dis], dep, n_train)

#int
X_cont, X_dis, y = items_to_arrays(items_d)
X_cont.shape, X_dis.shape, y.shape

assert np.issubdtype(X_dis.dtype, np.integer), 'discrete array dtype should be an np.integer subtype'

(200000, 4, 10) (200000, 2, 10)


In [ ]:
a = np.random.randn(2,10)

In [ ]:
a.shape, a[None].shape, a[:, None].shape

((2, 10), (1, 2, 10), (2, 1, 10))

In [ ]:
type(dsets.items[0][0])

numpy.ndarray

NOTE: slicing into the torchtools tensory types does not preserve the type but casts them to pytorch tensors
tsai subtypes define __getitem__ like below for that reason
```
def __getitem__(self, idx):
        res = super().__getitem__(idx)
        return res.as_subclass(type(self))
```

In [ ]:
class TSDatasets(NumpyDatasets):
    "A dataset that creates tuples from X (and y) and applies `item_tfms`"
    _xtype, _xdistype, _ytype = TSTensor, TSIntTensor, None # Expected X and y output types (torch.Tensor - default - or subclass)
    def __init__(self, X=None, X_dis=None, y=None, items=None, sel_vars=None, sel_steps=None, tfms=None, tls=None, n_inp=None, dl_type=None,
                 inplace=True, **kwargs):
        self.inplace = inplace
        if tls is None:
            X = itemify(to3darray(X), tup_id=0)
            X_dis = itemify(to3darray(X_dis), tup_id=0) if X_dis is not None else X_dis
            y = itemify(y, tup_id=0) if y is not None else y
            items = tuple((X,)) if y is None else tuple((X,y))
            if X_dis is not None: items = tuple((X, X_dis, y)) if y is not None else tuple(X, X_dis,)
            self.tfms = L(ifnone(tfms,[None]*len(ifnone(tls,items))))
            
#         if X_dis is not None: self.X_dis = X_dis
       
        self.sel_vars = ifnone(sel_vars, slice(None))
        self.sel_steps = ifnone(sel_steps,slice(None))
        self.tls = L(tls if tls else [TfmdLists(item, t, **kwargs) for item,t in zip(items,self.tfms)])
        self.n_inp = (1 if len(self.tls)==1 else len(self.tls)-1) if n_inp is None else n_inp
        if len(self.tls[0]) > 0:
            _tls_types = [self._xtype, self._ytype] if len(self.tls)==2 else [self._xtype, self._xdistype, self._ytype]
            print(_tls_types)
#             print(len(self.tls))
#             for tl,_typ in zip(self.tls, _tls_types):
#                 print (len(tl), _typ, type(tl[0]), isinstance(tl[0], torch.Tensor))
            self.types = L([ifnone(_typ, type(tl[0]) if isinstance(tl[0], torch.Tensor) else tensor) for 
                            tl,_typ in zip(self.tls, _tls_types)])
            self.ptls = L([tl if not self.inplace else tl[:] if type(tl[0]).__name__ == 'memmap' else 
                           tensor(stack(tl[:])) for tl in self.tls])

    def __getitem__(self, it):
        
#         for i,(ptl,typ) in enumerate(zip(self.ptls,self.types)):
#             print (i, typ)
        
#         return tuple([typ(ptl[it])[...,self.sel_vars, self.sel_steps] if i==0 else 
#                       typ(ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])
        ## do not enable slicing for now 
        return tuple([typ(ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])
    

    def subset(self, i): return type(self)(tls=L(tl.subset(i) for tl in self.tls), n_inp=self.n_inp, 
                                           inplace=self.inplace, tfms=self.tfms,
                                           sel_vars=self.sel_vars, sel_steps=self.sel_steps)
    @property
    def vars(self): return self[0][0].shape[-2]
    @property
    def len(self): return self[0][0].shape[-1]


In [ ]:
splits = FixedSplitter()(df_main)
list(map(len,splits))

[10000, 5000]

In [ ]:
## smaller ground dataset
x1, x2, y2 = X_cont[:20000], X_dis[:20000], y[:20000]

NameError: name 'X_cont' is not defined

In [ ]:
tfms=None
dsets_d = TSDatasets(X=x1, X_dis=x2, y=y2, tfms=tfms, splits=FixedSplitter()(df_main), inplace=True)

In [ ]:
dsets_d.tfms

(#3) [None,None,None]

In [ ]:
norm = TSNormalize(by_sample=False, by_var=True)
tszero = TimestepZero(verbose=True)

In [ ]:
@Augmix
def encodes(self, x:TSIntTensor):
    return cast(self.__call__(TSTensor(x.float())).int(), type(x))

In [ ]:
augmix = Augmix(verbose=True)

tfms None


In [ ]:
augmix.tfms

[YWarp: (TSIntTensor,object) -> encodes
 (TSTensor,object) -> encodes , YNormal: (TSIntTensor,object) -> encodes
 (TSTensor,object) -> encodes , YScale: (TSIntTensor,object) -> encodes
 (TSTensor,object) -> encodes , TimeWarp: (TSIntTensor,object) -> encodes
 (TSTensor,object) -> encodes , TimeNormal: (TSTensor,object) -> encodes ]

In [ ]:
xd[0]

tensor([[ 0,  0,  0,  3,  1,  2, -1, -1,  1,  0],
        [ 2, -2,  0, -3,  3, -2, -2,  2,  1, -1]], device='cuda:0')

In [ ]:
xd

TSIntTensor([[[ 0,  0,  0,  ..., -1,  1,  0],
         [ 2, -2,  0,  ...,  2,  1, -1]],

        [[ 1,  0, -2,  ...,  0, -3,  5],
         [ 2,  0,  0,  ..., -2,  0, -3]],

        [[-1,  3, -3,  ..., -1,  1,  1],
         [-1,  1,  0,  ..., -1,  0,  1]],

        ...,

        [[ 1,  1,  2,  ...,  0,  0,  0],
         [-2, -1, -2,  ...,  1,  0,  2]],

        [[-2,  0, -3,  ...,  0,  0,  1],
         [-2,  1,  0,  ...,  0,  0,  4]],

        [[ 0,  0,  0,  ...,  0,  2,  3],
         [ 0,  0,  2,  ...,  1,  0, -3]]], device='cuda:0')

In [ ]:
augmix(xd)[0]

weights [0.39461015 0.28242359 0.32296627], m 0.2661396293905798
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
timewarp
ywarp
timenormal torch.Size([256, 2, 10])
ops in step 1 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
yscale
ops in step 2 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp


tensor([[ 0,  0,  0,  2,  1,  1,  0, -1,  0,  0],
        [ 1, -1,  0, -2,  2, -1, -2,  1,  0,  0]], device='cuda:0',
       dtype=torch.int32)

In [ ]:
dls_d = TSDataLoaders.from_dsets(dsets_d.train, dsets_d.valid, bs=[256, 512], num_workers=0, 
                              after_batch=Pipeline(noop),
                            batch_tfms=[norm, ToTT, augmix])

In [ ]:
xc, xd, y = dls_d[0].one_batch()

weights [0.50549484 0.46092898 0.03357618], m 0.49902090803471816
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
ops in step 1 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
ops in step 2 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
weights [0.85618547 0.053768   0.09004653], m 0.6764267086731014
ops in step 0 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
ynormal
yscale
timenormal torch.Size([256, 2, 10])
ops in step 1 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
ynormal
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
ynormal
timenormal torch.Size([256, 2, 10

In [ ]:
xc.shape, xd.shape, y.shape

(torch.Size([256, 4, 10]), torch.Size([256, 2, 10]), torch.Size([256]))

In [ ]:
# #export
# class InceptionTimeD(nn.Module):
#     '''
#     add a sigmoid layer to InceptionTime to get the ouput in a certain range
#     '''
    
#     def __init__(self, n_in, n_out):
#         super().__init__()
#         self.mod = nn.Sequential(InceptionTime(n_in, n_out), Sigmoid(-1., 1.))
        
#     def forward(self, xc, xd):
#         x = torch.cat([xc.float(), xd.float()], dim=-2)
#         x = x.float()
#         print(f'InceptionTimeSgm dtype {x.dtype}')
#         return self.mod(x)

In [ ]:
model = InceptionTimeD(6,1)
loss_fn = get_loss_fn('leaky_loss', alpha=0.5)

In [ ]:
learn = Learner(dls_d, model, loss_fn, metrics=[unweighted_profit, partial(unweighted_profit, threshold=0.5)])

In [ ]:

learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,unweighted_profit,unweighted_profit,time
0,0.084339,-0.503737,0.287363,0.287363,00:04
1,0.048007,-0.388225,-0.399168,0.221982,00:04
2,-0.102091,-0.125452,0.039632,0.039632,00:04
3,-0.218451,-0.419497,0.221982,0.221982,00:04
4,-0.361746,-0.419235,0.221982,0.221982,00:04


weights [0.15487448 0.32985151 0.51527401], m 0.2509213468625352
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
ops in step 1 [TimeNormal: (TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
ynormal
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
timenormal torch.Size([256, 4, 10])
timewarp
weights [0.44564088 0.0129572  0.54140192], m 0.9389955749991228
ops in step 0 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
yscale
timewarp
ops in step 1 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
yscale
ynormal


weights [0.00139337 0.9154946  0.08311203], m 0.6332814953319451
ops in step 0 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
timewarp
yscale
ops in step 1 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
timewarp
ywarp
timenormal torch.Size([256, 4, 10])
ops in step 2 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
ywarp
weights [0.19412081 0.51541526 0.29046393], m 0.7305892617863522
ops in step 0 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes 

weights [0.70201996 0.20149892 0.09648113], m 0.3577606331321787
ops in step 0 [TimeNormal: (TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
ywarp
ops in step 1 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
yscale
ywarp
ops in step 2 [TimeNormal: (TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
timewarp
weights [0.32557258 0.34329688 0.33113055], m 0.525389511773879
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
timewarp
ywarp
timenormal torch.Size([256, 2, 10])
ops in step 1 [YWarp: (TSInt

weights [0.15303745 0.16623991 0.68072264], m 0.7866409216089238
ops in step 0 [TimeNormal: (TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
ynormal
timewarp
ops in step 1 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
timenormal torch.Size([256, 4, 10])
ywarp
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
weights [0.29969472 0.45977712 0.24052816], m 0.6777128321722152
ops in step 0 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
timewarp
ops in step 1 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TST

...

weights [0.09847321 0.89510694 0.00641986], m 0.07488777352739777
ops in step 0 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
ops in step 1 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
weights [0.22113665 0.51511252 0.26375083], m 0.7669278339341468
ops in step 0 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
ops in step 1 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
yscale
ops in step 2 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
InceptionTimeSgm dtype torch.float32
weights [0.09753758 0.41064537 0.49181705], m 0.8835288176241481
ops in step 0 [TimeNormal: (TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> en

weights [0.571009   0.34159544 0.08739556], m 0.6437062249865199
ops in step 0 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
ops in step 1 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
ynormal
ywarp
timenormal torch.Size([256, 4, 10])
ops in step 2 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
ynormal
yscale
weights [0.27741624 0.01316976 0.709414  ], m 0.6604977472477636
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
ops in step 1 [TimeNormal: (TSTensor,object) -> encodes ]
timenormal torch.Size([256, 2, 10])
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]


weights [0.42730533 0.47179161 0.10090306], m 0.5703502931629647
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
timewarp
timenormal torch.Size([256, 4, 10])
ops in step 1 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
ynormal
timenormal torch.Size([256, 4, 10])
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
timenormal torch.Size([256, 4, 10])
yscale
weights [0.50183157 0.49362334 0.00454509], m 0.33700033680947944
ops in step 0 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
yscale
ops in step 1 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,o

weights [0.74107724 0.12139491 0.13752785], m 0.0554919196350367
ops in step 0 [TimeNormal: (TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
ynormal
yscale
ops in step 1 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
ywarp
timenormal torch.Size([256, 4, 10])
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
timewarp
weights [0.0039724  0.81297147 0.18305612], m 0.6082991511566817
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
timewarp
timenormal torch.Size([256, 2, 10])
ops in step 1 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
ops in st

weights [0.6368481  0.26360939 0.0995425 ], m 0.14157664836880587
ops in step 0 [TimeNormal: (TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
ynormal
yscale
ops in step 1 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
yscale
ops in step 2 [TimeNormal: (TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
yscale
weights [0.15578422 0.83091898 0.0132968 ], m 0.42643678995168005
ops in step 0 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
timewarp
ops in step 1 [TimeNormal: (TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 2, 10])
timewarp
ops in 

In [ ]:
preds = learn.get_preds()

InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32


In [ ]:
unweighted_profit(*preds)

tensor(0.2220)

In [ ]:
(preds[0]>0).sum()

tensor(302)

In [ ]:
preds[0][:40]

tensor([[-1.0000],
        [-1.0000],
        [-1.0000],
        [-1.0000],
        [-1.0000],
        [-1.0000],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [ 1.0000],
        [-1.0000],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915]])

In [ ]:
544*520

282880

In [ ]:
540*523

282420